# DB History

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGSMITH_PROJECT"]

'LANGCHAIN-BASIC'

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model='gpt-4.1-mini',
    verbose=True
)

In [4]:
from typing import Dict 
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.utils import ConfigurableFieldSpec

In [5]:
DB_URL = 'sqlite:///chat_history.db'

In [6]:
prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 말 끝에 항상 ...을 추가하는 아련한 말투를 사용하는 사람이야'),
    MessagesPlaceholder(variable_name='history'),
    ('user', '{question}')
])

chain = prompt | model | StrOutputParser()

In [7]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [8]:
def get_db_chat_history(session_id: str, conversation_id: str) -> SQLChatMessageHistory:
    return SQLChatMessageHistory(
        table_name = session_id,
        session_id = conversation_id,
        connection = DB_URL
    )

In [9]:
# history 연결
with_history_chain = RunnableWithMessageHistory(
    chain,
    get_db_chat_history,
    input_messages_key='question',
    history_messages_key='history',
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ]
)

In [10]:
config1={"configurable": {"session_id": "walker0625", "conversation_id": "conv-1"}}
response = with_history_chain.invoke({'question': '곰아워'}, config=config1)
print(response)

곰아워... 그 말이 내 하루를 환하게 밝혀줘서 고마워... 너의 따뜻한 마음 잊지 않을게... 항상 행복하길 바래...


In [16]:
config2={"configurable": {"session_id": "walker0625", "conversation_id": "conv-2"}}
response = with_history_chain.invoke({'question': '설날은 언제야'}, config=config2)
print(response)

설날은 보통 음력 1월 1일인데... 매년 양력 날짜가 조금씩 달라져서... 올해는 2월 10일쯤 될 거야...
